# Sequences

There's lots of data that takes the form of variable length sequences. Think about:
* stock prices
* weather data
* fishery and farming yield data
* health measurements (your smart watch!)
* speech data
* text data
* music data

When dealing with sequences, you may want to:
1. Classify the whole sequence (sequence in, one label out)
2. Classify things in the sequence (sequence in, sequence out)
3. Generate more steps in the sequence (sequence in, continuation sequence out)

We are going to use case 3 as the running example here. So, for example, you have a history of temperatures on Colby's campus and you want to know what tomorrow's temperature will be; or, you have the text "Wherefore art" and you want to know what should come next. In the NLP context, these are called *language models*. 

A language model estimates the joint probability of a sequence. Assuming the input consists of data points $x_1, x_2, ..., x_t$, the language model tracks $P(x_1, x_2, ..., x_t)$. We can decompose this into $P(x_1) \prod_{t=2}^T P(x_t|x_{t-1}, ..., x_t)$. Once we have a model like this we can:
* Given a $x_1, ..., x_t$, evaluate its likelihood.
* Sample a new sequence $x_1, ..., x_t$.
* Predict the next items in the sequence. Here, you want to know for all possible $x_{t+1}$, $P(x_{t+1}|x_t, ..., x_1)$. (And after that, you probably want to repeat for $x_{t+2}$ and so on!)

A model that regresses the value of a signal on the previous values of that signal is called an *autoregressive model*. Generally there are two ways to do autoregressive modeling:
1. Approximate the variable-length sequence (history) with fixed-length subsequences.
2. Approximate the variable-length sequence (history) with a fixed-length summary $h_t$ of the history.

# Tokenizing text data

Caveat: text data doesn't come pre-tokenized, so you have to tokenize it.

Tokenization is a critical step:
* it has to be *consistent* across training, dev and test data
* it has to produce an (ideally small) fixed vocabulary of tokens
* it has to retain information essential to give predictive power for the task at hand

Common approaches to tokenization include:
1. Character-level (or byte-level, in the case of CJKV languages) - this gives a small fixed vocabulary, at some cost in efficiency and at the cost of radically increasing the predictive context necessary
2. "Word" level - this gives a largeish fixed vocabulary and an unknown token problem, but you can be more efficient in prediction and the models tend to be easier for humans to understand
3. Byte-pair encoding - a byte-pair encoding is a very efficient representation of text data, originally developed for data compression. To get the vocabulary for byte-pair encoding you repeatedly walk over the input text, replacing the most common character subsequences with a new fake character. You do this until there aren't very many common character subsequences left. This is effective for data compression because of the nature of text, which tends to follow Zipf's law.

For developing an understanding of RNNs and for the next project, we will use character-level tokenization.

# Approximating variable-length sequences with fixed-length subsequences, approach one: count!

The simplest way to deal with this is just to walk over your variable-length sequence, taking subsequences of fixed length $n$. Then you can use relative frequencies as probability estimates.

One of the simplest approaches to language modeling is implemented in the srilm toolkit (http://www.speech.sri.com/projects/srilm/). You tokenize your training data, then you walk over it extracting *ngrams* (sequences of length 1, or 2, or 3, or 4, or 5). You estimate the probability of each ngram using its relative frequency.

Consider the small language with three characters $a, b, c$ and training data $abcbabcbabcbabc$. Let's say we want to develop a bigram language model (ngram of two). We extract all subsequences of length two and count them up:

| ngram | count |
| ----- | ----- |
| ab | 4 |
| bc | 4 |
| cb | 3 |
| ba | 3 |

Then, we can estimate the probability of each sequence using relative frequencies.

One issue with this type of approach is if you want to predict next items in a sequence. For each next item you predict, the model introduces some error, so over time the predictions will be increasingly inaccurate. 

Another issue with this type of approach is that you may come across subsequences at inference time that you never saw at training time (and therefore don't have a probability estimate for). One way to deal with this is using *Laplace smoothing*: add a small constant to all counts (and make sure to add the sum of all the small constants in the denominator of the relative frequencies!). 

A third issue with this type of approach is that it doesn't extend well to large contexts (large histories). Therefore, we prefer to train a neural network language model rather than build a count-based language model (see https://arxiv.org/pdf/1606.00499.pdf).

By the way, if this section feels familiar it should! If you've taken CS251/2 you saw a bunch of this when you learned about naive Bayes.



# Approximating variable-length sequences with fixed-length subsequences, approach two: predict!

We can do better than count-based methods for language modeling using any one of a number of machine-learning methods. But in order to use those, we first have to represent text input as numbers.

## Representing text data

Here are three types of representation that have been used:
1. One-hot encoding - each token gets an index $i$ in an array of length $n$. When it appears, its representation is a $n$-length feature with the $ith$ value being 1.
2. Feature vector - each token gets a feature vector, for example including its length, its "word shape", its word class (part of speech tag), its tf-idf value....
3. Fixed word embedding - first, take a large corpus of text and calculate (or train, using a simple neural network) a $m$ length *word embedding* for each token in the text. That word embedding is the numeric representation of each token.

Of course, it's generally impossible to fix ahead of time all the tokens you might see in a text, so you have to also have a vector for <unk>. 

For developing an understanding of RNNs and for the next project, we will use one-hot encoding.

# Modeling

Once you have a vector representation for each token in a text, then you can feed those directly into a MLP or a CNN. 

The well-known spacy library for NLP does exactly this (and for all three types of sequence task!): for an input, it first runs "tok2vec", which builds a vector representation for each input token; then it uses a simple linear layer or a CNN for downstream processing.

# Evaluation of language models

The standard metric for evaluating a language model is perplexity. Perplexity is a measure related to entropy.

Think of a language model as a classification model over sequences, with the labels being the next token in the sequence. We use cross-entropy loss for classification. Consider the cross-entropy loss averaged over all the tokens in a sequence:
$\frac{1}{n} \sum_{t=1}^N -log P(x_t|x_{t-1}, ..., x_1)$

Perplexity is just the exponential of this average cross entropy loss.
* *What is the perplexity of a model that works perfectly?*
* *What is the perplexity of a model that is always wrong?*
* *What is the perplexity of a model that predicts the next token at random?*

# The next project

For the next project, you will work with a small dataset of sequence data. You will fit a MLP to this data using fixed-length subsequences. You will implement a RNN and fit the RNN to this data. You will compare the performance of the two models using perplexity.